In [ ]:
!pip install GPUtil

import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU not available.")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=93391bf30b74599d6cfbb3ac0e4f84f3456aa8ba1cf171ccf373d58de6e1d801
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available!


In [ ]:
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datetime import datetime

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `finetune` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `finetune`


In [ ]:
base_model_id = "meta-Llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
train_dataset = load_dataset("csv", data_files="dataset.csv", split="train")
train_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 325
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-Llama/Llama-2-7b-chat-hf")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'])
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

In [ ]:
model.gradient_checkpointing_enable() # Bật gradient checkpointing, chỉ lưu nhưng đầu ra quan trọng
model = prepare_model_for_kbit_training(model) # Chuẩn bị mô hình cho huấn luyện với k-bit precision

config = LoraConfig(
    r=8, # Số rank thấp cho các ma trận được thêm vào.
    lora_alpha=64, # Hệ số khuếch đại (scaling factor) của LoRA.
    target_modules=[  # Các lớp mô hình cần áp dụng LoRA.
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
        "o_proj"
    ],
    bias="none", # Không áp dụng LoRA cho các bias.
    lora_dropout=0.05, # Tỷ lệ dropout để tránh overfitting.
    task_type="CAUSAL_LM", # Loại tác vụ: "Causal Language Modeling".
)

model = get_peft_model(model, config) # Kết hợp mô hình gốc với cấu hình LoRA bằng cách sử dụng PEFT framework.

In [ ]:
trainer = transformers.Trainer(
    model=model, # Mô hình sử dụng tinh chỉnh
    train_dataset=tokenized_dataset, # Bộ dataset sử dụng tinh chỉnh
    args=transformers.TrainingArguments(
        output_dir="./finetuned-llama-2-model-TuyenSinhPTIT2024",  # Thư mục lưu lại các checkpoint huấn luyện
        per_device_train_batch_size=2, # Số mẫu dữ liệu mỗi bath cho GPU
        gradient_accumulation_steps=2, # Tích lũy gradient sau 2 bước trước khi thực hiện cập nhật trọng số
        num_train_epochs=3, # Số lần duyệt qua toàn bộ dữ liệu huấn luyện
        learning_rate=2e-4, # Xác định mức độ thay đổi trọng số trong mỗi bước cập nhật
        bf16=False,      # Tắt định dạng bfloat16 do phần cứng không hỗ trợ
        optim="paged_adamw_8bit", # Sử dụng tối ưu hóa Adamw phiên bản 8bit để tiết kiệm bộ nhớ và tăng hiệu suất
        logging_dir="./logs",# Thư mục sử dụng để lưu thông tin huấn luyện
        save_strategy="epoch", # Lưu lại model huấn luyện sau mỗi epoch
        logging_steps = 10 # Ghi lại thông tin nhật kí huấn luyện
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # Tắt cache trong bộ huấn luyện để tối ưu bộ nhớ
trainer.train() # Tiến hành huấn luyện

trainer.save_model("./finetuned_llama2_7b_chat")
tokenizer.save_pretrained("./finetuned_llama2_7b_chat")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.339300
20,1.047000
30,0.955600
40,0.857900
50,0.814100
60,0.740800
70,0.834100
80,0.695200
90,0.592100
100,0.569400


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('./finetuned_llama2_7b_chat/tokenizer_config.json',
 './finetuned_llama2_7b_chat/special_tokens_map.json',
 './finetuned_llama2_7b_chat/tokenizer.model',
 './finetuned_llama2_7b_chat/added_tokens.json',
 './finetuned_llama2_7b_chat/tokenizer.json')

In [ ]:
trainer.model.save_pretrained("newmodel")
tokenizer.save_pretrained("newmodel")

('newmodel/tokenizer_config.json',
 'newmodel/special_tokens_map.json',
 'newmodel/tokenizer.model',
 'newmodel/added_tokens.json',
 'newmodel/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-Llama/Llama-2-7b-chat-hf"

base_model = AutoModelForCausalLM.from_pretrained( # Tải lại model gốc bản đầy đủ
    base_model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained('meta-Llama/Llama-2-7b-chat-hf') # Tải lại tokenizer

modelFinetuned = PeftModel.from_pretrained(base_model,"newmodel") # Load adapter vào model gốc
modelFinetuned = modelFinetuned.merge_and_unload() # Megre model gốc với adapter sinh ra model mới

model.save_pretrained("llama2-7b-chat-TuyenSinhPTIT2024") # Lưu lại mô hình tại local
tokenizer.save_pretrained("llama2-7b-chat-TuyenSinhPTIT2024") # Lưu lại tokenizer tại local

model.push_to_hub("llama2-7b-chat-TuyenSinhPTIT2024", use_temp_dir=False) # Đẩy mô hình lên HuggingFace
tokenizer.push_to_hub("llama2-7b-chat-TuyenSinhPTIT2024", use_temp_dir=False)# Đảy tokenizer HuggingFace